# 1 Imports

In [49]:
import os
import requests
from dotenv import  load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# 2 Connect to OpenAI

In [50]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPEN_API_KEY')

In [51]:
openai = OpenAI()

In [52]:
class Website:
    """
    A utility class to represent a Website that we have scrapped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url the BeutifulSoup library
        """

        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"

        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        self.text = soup.body.get_text(separator="\n", strip=True)

In [64]:
website = Website("https://edwarddonner.com")
print(website.title)
print(website.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

# 3 Prompt Configuration

In [54]:
# Define system prompt

system_prompt = "You are an assistant that analyzes the contets of a website \
    and provides a short summary, ignoring text that might be navigation related. \
    Respond in markdown."

In [55]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are lookin at a website title {website.title}"
    user_prompt += """ The contents of this website is as follows; \
        please provide a short summary of this website in markdown. \
        If it includes news or annoucements, then summrarize these too.\n\n """
    user_prompt += website.text
    return user_prompt

In [56]:
# OpenAI Message API structure

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

# 4 OpenAI API Call

## 4.1 Summary Edwarddonner Website

In [57]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content       

In [59]:
output = summarize("http://edwarddonner.com")

In [60]:
# A function to display this nicely in Jupyter output, using Markdown
def display_summary(text):
    display(Markdown(text))

In [61]:
display_summary(output)

# Summary of Edward Donner's Website

Edward Donner's website serves as a personal space where he shares his interests in coding, experimenting with large language models (LLMs), and amateur electronic music production. As the co-founder and CTO of Nebula.io, he focuses on leveraging AI to enhance talent discovery and management in recruitment. Previously, he founded the AI startup untapt, which was acquired in 2021.

## Key Features
- **Connect Four**: A platform enabling LLMs to compete in a game of strategy and diplomacy.
- **Outsmart**: Another arena for LLM competitions.
- **Resource Posts**: The website features several announcements related to workshops and resources for AI and LLM engineering.

## Recent News and Announcements
- **January 23, 2025**: LLM Workshop – Hands-on with Agents – resources released.
- **December 21, 2024**: Welcome message to SuperDataScientists.
- **November 13, 2024**: Resources for Mastering AI and LLM Engineering shared.
- **October 16, 2024**: Resources for transitioning from Software Engineer to AI Data Scientist provided.

## 4.2 Summary of CNN Website

In [65]:
display_summary(summarize("http://cnn.com"))

# CNN Website Summary

CNN's website provides a comprehensive array of breaking news headlines, articles, and video content covering various topics, including current events, politics, business, health, and global affairs. Key recent stories include:

- **Kash Patel** confirmed as the new FBI director, highlighting his ties to former President Trump and raising questions about implications for law enforcement.
- Recent CNN polls indicate that Trump’s approval rating is at a peak, though it may not be sustainable.
- The **Hamas** group has conducted a ceremonial release of bodies of Israeli hostages, indicating ongoing tensions in the region.
- Further global news covers significant events like the recovery of illegal timber in Brazil and insights into the aftermath of conflicts affecting voters in Germany.

The site also features diverse sections dedicated to sports, entertainment, technology, and lifestyle, alongside various programs, analyses, and specialized newsletters aimed at keeping readers informed on specific topics of interest. The platform encourages user interaction and feedback regarding its content and advertisements.

## 4.3 Summary of InfoMoney Website

In [66]:
display_summary(summarize("http://infomoney.com.br"))

# InfoMoney – Informações que valem dinheiro

InfoMoney é uma plataforma dedicada a fornecer informações financeiras e de investimento, abordando diversos tópicos relevantes tanto para investidores iniciantes quanto experientes. O site oferece cursos gratuitos, simuladores de investimentos, planilhas e ferramentas que ajudam os usuários a gerenciar suas finanças pessoais e tomar decisões informadas sobre onde investir.

## Últimas Notícias

- **Bolsa de Valores**: O mercado fechou em leve alta, impulsionado pela ação da Vale, enquanto bancos como o BB e a Gerdau enfrentaram quedas significativas.
- **Câmbio**: O dólar teve uma queda, cotando a R$ 5,70, influenciado por tendências do mercado externo.
- **Resultados Financeiros**: A Renner reportou um lucro líquido de R$ 487,2 milhões, o que representa uma queda de 7,5% em relação ao ano anterior.
- **Análise de Ações**: Apesar dos prejuízos, a Vale teve um aumento de 3,7% em suas ações, gerando otimismo entre os investidores.

### Outros Tópicos Relevantes
- **Mercados**: BB teve lucros em linha com as expectativas, mas suas ações caíram 3% devido a fatores externos. Gerdau também relatou resultados em linha, mas com desafios iminentes para 2025.
- **Política**: O STF mantém um prazo para a defesa de Bolsonaro em relação a denúncias da PGR.
- **Economia**: Nubank apresentou um lucro líquido ajustado de US$ 610 milhões, marcando um aumento de 87% em comparação ao ano passado.

InfoMoney continua a ser uma fonte crucial para informações financeiras, análises de mercado e tendências econômicas, ajudando o público a navegar pelo cenário financeiro em constante mudança.